In [4]:
###############################
#Step 1: Read in turnstile data

import pandas as pd

df = pd.read_csv('turnstile.csv')
mydict={}

df.columns = df.columns.str.strip()

df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/10/2017,00:00:00,REGULAR,6215258,2104297
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/10/2017,04:00:00,REGULAR,6215284,2104303
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/10/2017,08:00:00,REGULAR,6215318,2104337
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/10/2017,12:00:00,REGULAR,6215475,2104417
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/10/2017,16:00:00,REGULAR,6215841,2104465


In [9]:
################################
#Step 2: Create a dictionary mapping (C/A,UNIT,Station,Date) to all other info
mydict = {}

for i in range(len(df.index)):
    mytuple = (df['C/A'][i], df.UNIT[i], df.STATION[i], df.DATE[i])
    
    if mytuple not in mydict.keys():
        mydict[mytuple] = [[df.SCP[i],df.LINENAME[i],df.DIVISION[i], df.TIME[i],df.DESC[i],df.ENTRIES[i],df.EXITS[i]]]
    else:
        mydict[mytuple].append([df.SCP[i],df.LINENAME[i],df.DIVISION[i] ,df.TIME[i],df.DESC[i],df.ENTRIES[i], df.EXITS[i]])

mydict[('A002', 'R051', '59 ST', '06/10/2017')][0:5]

[['02-00-00', 'NQR456W', 'BMT', '00:00:00', 'REGULAR', 6215258, 2104297],
 ['02-00-00', 'NQR456W', 'BMT', '04:00:00', 'REGULAR', 6215284, 2104303],
 ['02-00-00', 'NQR456W', 'BMT', '08:00:00', 'REGULAR', 6215318, 2104337],
 ['02-00-00', 'NQR456W', 'BMT', '12:00:00', 'REGULAR', 6215475, 2104417],
 ['02-00-00', 'NQR456W', 'BMT', '16:00:00', 'REGULAR', 6215841, 2104465]]

In [11]:
################################
#Step 3: Create a dictionary mapping (C/A,UNIT,Station,Date) 
#to an inner dictionary of (turnstile number, hourly rider entries or exits) key-value pairs

ent_or_exit = 6 # Choose 5 for entry, 6 for exit
dictwithin={}
newdict={}

for key in mydict.keys():
    dictwithin={}
    for i in range(len(mydict[key])):
        if mydict[key][i][0] not in dictwithin:
            dictwithin[mydict[key][i][0]]=[mydict[key][i][ent_or_exit]]
        else:
            dictwithin[mydict[key][i][0]].append(mydict[key][i][ent_or_exit])
    newdict[key] = dictwithin

newdict[('A002', 'R051', '59 ST', '06/10/2017')]

{'02-00-00': [2104297, 2104303, 2104337, 2104417, 2104465, 2104499],
 '02-00-01': [1247142, 1247146, 1247157, 1247209, 1247252, 1247285],
 '02-03-00': [3434712, 3434730, 3434800, 3435115, 3435421, 3435696],
 '02-03-01': [509875, 509906, 509982, 510268, 510601, 510948],
 '02-03-02': [7436819, 7436839, 7436914, 7437130, 7437365, 7437582],
 '02-03-03': [5950628, 5950642, 5950709, 5950885, 5951013, 5951128],
 '02-03-04': [3377741, 3377746, 3377792, 3377866, 3377911, 3377956],
 '02-03-05': [1315954, 1315958, 1315967, 1315993, 1316002, 1316018],
 '02-03-06': [482949, 482949, 482951, 482954, 482956, 482958],
 '02-05-00': [0, 0, 0, 0, 0, 0],
 '02-05-01': [738, 738, 738, 738, 738, 738],
 '02-06-00': [150963, 150967, 150968, 150972, 150975, 150981]}

In [12]:
################################
#Step 4: Create a dictionary mapping (C/A,UNIT,Station,Date) to total daily ridership of all turnstiles in
#that C/A,UNIT,Station location

totaldict={}

for key,val in newdict.items():
    count = 0
    for innerkey, innerval in val.items():
        for i in range(1,len(innerval)):
            count = count + innerval[i] - innerval[i-1]
    totaldict[key] = count

totaldict[('A002', 'R051', '59 ST', '06/10/2017')]


3971

In [21]:
################################
#Step 5: Create a dictionary mapping (C/A,UNIT,Station,Date) to the total ridership between two time values.
#Note that times [00:00:00, 04:00:00, 08:00:00, 12:00:00, 16:00:00, 20:00:00] correspond to indices [0,1,2,3,4,5]

timedict={}
time = 3 #(for between 8 and 12 pm)
#time = 5 #(for between 4 pm to 8 pm )
for key,val in newdict.items():
    count = 0
    for innerkey, innerval in val.items():
        try:
            count = count + innerval[time] - innerval[time-1]
        except:
            break
        timedict[key] = count

timedict[('A002', 'R051', '59 ST', '06/10/2017')]

1232

In [31]:
################################
#Step 6: Create a dictionary mapping (THE ENTIRE STATION, date) to total ridership

#if you want to do total DAILY ridership, use the totaldict dictionary
#if you want to do total HOURLY ridership between two times, use timedict

use_dict=timedict #or totaldict

station_date={}
for key, val in use_dict.items():
    if (key[2],key[3]) not in station_date:
        station_date[(key[2],key[3])] = use_dict[key]
    else:
        station_date[(key[2],key[3])] += use_dict[key]

print(station_date[('34 ST-HERALD SQ', '06/10/2017')])
        

15420


In [19]:
################################
#Step 7: Create a dictionary that maps each station to a list of tuples containing (date, total ridership)

all_station_dates = {}

for key,val in station_date.items():
    if key[0] not in all_station_dates:
        all_station_dates[key[0]]=[(key[1], station_date[key])]
    else:
        all_station_dates[key[0]].append((key[1], station_date[key]))
        
print(all_station_dates[('34 ST-HERALD SQ')])

[('06/10/2017', 15420), ('06/11/2017', 2038823441), ('06/12/2017', 37677), ('06/13/2017', 40329), ('06/14/2017', 35259), ('06/15/2017', 33189), ('06/16/2017', 37654)]


In [20]:
################################
#Step 8: Create a list of sorted tuples in decending order of (ridership, stationname)

#if saturday use 0, sunday use 1, monday use 2, etc.
weekday = 0

stationcount=[]
stationdate=[]

total_ridership_counts=[]

for station in all_station_dates.keys():
    if len(all_station_dates[station])==7: #a few stations have less than a full week of data
        if station != 'LACKAWANNA': #LACKAWANNA doesn't have standardized times
            total_ridership_counts.append((all_station_dates[station][weekday][1], station))
            
total_ridership_counts.sort(reverse=True)

for info in total_ridership_counts:
    print(info)
    
fout = open('output.csv', 'w')
for info in total_ridership_counts:
    fout.write(str(info[0])+'  '+str(info[1])+'\n')
fout.close()

(20016, '34 ST-PENN STA')
(16173, '14 ST-UNION SQ')
(15420, '34 ST-HERALD SQ')
(12191, 'CANAL ST')
(11545, 'TIMES SQ-42 ST')
(11238, '42 ST-PORT AUTH')
(10467, '23 ST')
(9441, '125 ST')
(9071, 'GRD CNTRL-42 ST')
(8488, '86 ST')
(8064, '59 ST COLUMBUS')
(7655, 'W 4 ST-WASH SQ')
(7648, '72 ST')
(6794, '59 ST')
(6559, 'FLUSHING-MAIN')
(6552, '96 ST')
(6167, 'PATH NEW WTC')
(6025, '14 ST')
(5819, 'CHAMBERS ST')
(5807, "B'WAY-LAFAYETTE")
(5803, '8 AV')
(5672, '28 ST')
(5473, 'BEDFORD AV')
(5140, 'FULTON ST')
(5076, 'ATL AV-BARCLAY')
(4972, 'THIRTY THIRD ST')
(4816, 'SOUTH FERRY')
(4532, '77 ST')
(4506, 'WHITEHALL S-FRY')
(4295, 'GRAND ST')
(4249, '7 AV')
(4165, '47-50 STS ROCK')
(4132, '116 ST')
(3976, 'JKSN HT-ROOSVLT')
(3972, '50 ST')
(3862, '42 ST-BRYANT PK')
(3408, 'DELANCEY/ESSEX')
(3169, '81 ST-MUSEUM')
(3143, 'LEXINGTON AV/53')
(3141, '145 ST')
(3033, '1 AV')
(2992, 'JAMAICA CENTER')
(2988, 'PRINCE ST')
(2962, '5 AV/59 ST')
(2900, '66 ST-LINCOLN')
(2872, '103 ST')
(2845, 'WALL ST')
(

In [23]:
###############
#Step 10: Calculate the maximum difference between Monday and Sat morning commutes in order to find
#business hubs instead of touristy hubs

monday={}
for line in open('monday_morning_exits.csv'):
    words=line.split('  ')
    monday[words[1].strip()]=words[0]

saturday={}
for line in open('saturday_morning_exits.csv'):
    words=line.split('  ')
    saturday[words[1].strip()]=words[0]
    
diff = []
for key,val in monday.items():
    try:
        diff.append((int(val) - int(saturday[key]), key))
    except:
        pass
    
diff.sort(reverse = True)

for station in diff:
    print(station)
    
fout = open('output2.csv', 'w')
for info in diff:
    fout.write(str(info[0])+'  '+str(info[1])+'\n')
fout.close()

(31473, '47-50 STS ROCK')
(26357, 'GRD CNTRL-42 ST')
(22257, '34 ST-HERALD SQ')
(21610, 'LEXINGTON AV/53')
(21455, 'TIMES SQ-42 ST')
(17642, '23 ST')
(16562, '42 ST-BRYANT PK')
(16330, 'WALL ST')
(15682, 'FULTON ST')
(12926, 'CHAMBERS ST')
(12748, '59 ST')
(11190, '50 ST')
(10732, 'BOWLING GREEN')
(9344, '51 ST')
(9046, 'CANAL ST')
(8966, '28 ST')
(8427, '34 ST-PENN STA')
(7861, '14 ST-UNION SQ')
(7694, '86 ST')
(7418, '14 ST')
(5840, '49 ST')
(5427, '5 AVE')
(4910, 'LEXINGTON AV/63')
(4888, '168 ST')
(4875, '59 ST COLUMBUS')
(4630, 'BROOKLYN BRIDGE')
(4301, '7 AV')
(3599, '33 ST')
(3390, '5 AV/53 ST')
(3254, 'GRAND ST')
(3133, 'COURT SQ')
(3057, '33 ST-RAWSON ST')
(3003, 'JAY ST-METROTEC')
(2999, "B'WAY-LAFAYETTE")
(2901, '72 ST-2 AVE')
(2897, '57 ST-7 AV')
(2711, 'RECTOR ST')
(2493, 'HOUSTON ST')
(2476, '34 ST-HUDSON YD')
(2461, 'JAMAICA CENTER')
(2417, 'SPRING ST')
(2381, 'ATL AV-BARCLAY')
(2341, '77 ST')
(2287, '1 AV')
(2223, '57 ST')
(2222, 'CONEY IS-STILLW')
(2145, 'HIGH ST')
(21

In [27]:
##################
#Step 11: Create a list of the top 27 tech firms and the top 10 volume subway differences

addresses = []

count = 0
for line in open('output2.csv'):
    if count < 10:
        words=line.split('  ')
        addresses.append(words[1].strip()+' subway station ny')
    count +=1

#Remove FULTON & CHAMBERS since there isn't tech nearby and replace with ASTOR PLACE AND 18 ST

addresses.remove('WALL ST subway station ny')
addresses.remove('CHAMBERS ST subway station ny')

addresses.append('ASTOR ST subway station ny')
addresses.append('18 ST subway station ny')

tech = ['AMAZON NYC', 'AOL NYC', 'APPLE NYC', 'APPNEXUS NYC', 'BLOOMBERG NYC', 'BLUE APRON NYC', 'BUZZFEED NYC', 'ETRADE NYC', 'ETSY NYC', 'FACEBOOK NYC', 'FRESH DIRECT NYC', 'GOOGLE NYC', 'INFORMATION BUILDERS NYC', 'LINKEDIN NYC', 'MEDIARADAR NYC', 'MICROSOFT NYC', 'OSCAR NYC', 'SALESFORCE NYC', 'SHUTTERSTOCK NYC', 'SPOTIFY NYC', 'TUMBLR NYC', 'TWITTER NYC', 'VICE MEDIA NYC', 'WEWORK NYC', 'YELP NYC', 'YEXT NYC', 'ZOCDOC NYC']

addresses = addresses + tech
addresses

['47-50 STS ROCK subway station ny',
 'GRD CNTRL-42 ST subway station ny',
 '34 ST-HERALD SQ subway station ny',
 'LEXINGTON AV/53 subway station ny',
 'TIMES SQ-42 ST subway station ny',
 '23 ST subway station ny',
 '42 ST-BRYANT PK subway station ny',
 'FULTON ST subway station ny',
 'ASTOR ST subway station ny',
 '18 ST subway station ny',
 'AMAZON NYC',
 'AOL NYC',
 'APPLE NYC',
 'APPNEXUS NYC',
 'BLOOMBERG NYC',
 'BLUE APRON NYC',
 'BUZZFEED NYC',
 'ETRADE NYC',
 'ETSY NYC',
 'FACEBOOK NYC',
 'FRESH DIRECT NYC',
 'GOOGLE NYC',
 'INFORMATION BUILDERS NYC',
 'LINKEDIN NYC',
 'MEDIARADAR NYC',
 'MICROSOFT NYC',
 'OSCAR NYC',
 'SALESFORCE NYC',
 'SHUTTERSTOCK NYC',
 'SPOTIFY NYC',
 'TUMBLR NYC',
 'TWITTER NYC',
 'VICE MEDIA NYC',
 'WEWORK NYC',
 'YELP NYC',
 'YEXT NYC',
 'ZOCDOC NYC']

In [28]:
##################
#Step 12: Get lat/long coordinates using Google API. 
#Store these values in the where.js file. 
# Open the where.html file in your browser to view the map

import urllib.request, urllib.parse, urllib.error
import json
    
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/json?'
    
mystring = 'myData = [\n'

for address in addresses:
    url = serviceurl + urllib.parse.urlencode({'address': address})
    uh = urllib.request.urlopen(url)
    data = uh.read().decode()
    js = json.loads(data)
    try:
        if 'NY' not in js['results'][0]['formatted_address']:
            print('BAD!!!!!!!!', address)
            continue
    except:
        print(print('BAD!!!!!!!!', address))
        
    try:
        lat = js["results"][0]["geometry"]["location"]["lat"]
        lng = js["results"][0]["geometry"]["location"]["lng"]
        
        mystring = mystring + '['+str(js["results"][0]["geometry"]["location"]["lat"])+',' \
        +str(js["results"][0]["geometry"]["location"]["lng"])+', '+"'"+address+"'"+"],"
        print(address)
    except:
        print(print('BAD!!!!!!!!', address))
    
mystring = mystring[0:-1]
mystring = mystring + "];"                                                                                      

with open('where.js', 'w') as outfile:
     outfile.write(mystring)                                                                                          
outfile.close()

47-50 STS ROCK subway station ny
GRD CNTRL-42 ST subway station ny
34 ST-HERALD SQ subway station ny
LEXINGTON AV/53 subway station ny
TIMES SQ-42 ST subway station ny
23 ST subway station ny
42 ST-BRYANT PK subway station ny
FULTON ST subway station ny
ASTOR ST subway station ny
18 ST subway station ny
AMAZON NYC
AOL NYC
APPLE NYC
APPNEXUS NYC
BLOOMBERG NYC
BLUE APRON NYC
BUZZFEED NYC
ETRADE NYC
ETSY NYC
FACEBOOK NYC
FRESH DIRECT NYC
GOOGLE NYC
INFORMATION BUILDERS NYC
LINKEDIN NYC
MEDIARADAR NYC
MICROSOFT NYC
OSCAR NYC
SALESFORCE NYC
SHUTTERSTOCK NYC
SPOTIFY NYC
TUMBLR NYC
TWITTER NYC
VICE MEDIA NYC
WEWORK NYC
YELP NYC
YEXT NYC
ZOCDOC NYC
myData = [
[40.7578276,-73.98179069999999, '47-50 STS ROCK subway station ny'],[40.7527262,-73.9772294, 'GRD CNTRL-42 ST subway station ny'],[40.750082,-73.98813129999999, '34 ST-HERALD SQ subway station ny'],[40.7571494,-73.9721498, 'LEXINGTON AV/53 subway station ny'],[40.7556158,-73.9863347, 'TIMES SQ-42 ST subway station ny'],[40.7415465,-73.988